In [1]:
import numpy as np
import re
import nltk
import pandas as pd
#nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics 

In [2]:
recommendation = pd.read_csv('/Users/simrankumarisah/Downloads/feedbacksFormatted.csv',encoding='unicode_escape')
X, y = recommendation.Data, recommendation.Target

In [3]:
def sentimentClassification(text):
    blob = TextBlob(text)
    if blob.sentiment.polarity < 0:
        a="Not satisfied"
    elif blob.sentiment.polarity == 0:
        a="No Comment"
    elif blob.sentiment.polarity > 0 and blob.sentiment.polarity <0.6:
        a="Satisfied"
    else:
        a="Very Satisfied"
    return a,blob.sentiment.polarity

In [4]:
def feedback_classification(feedback):
    if feedback=="Behaviour torwards students":
        a="Behaviour torwards students"
    elif feedback=="Teaching Skills":
        a="Teaching Skills"
    elif feedback=="General":
        a="General"
    else:
        a="Punctuality"
    return a

In [5]:
def processed_documents(data):
    documentss = []
    from nltk.stem import WordNetLemmatizer
    stemmer = WordNetLemmatizer()
    for sen in range(0, len(data)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(data[sen]))
    
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
    
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
    
        # Converting to Lowercase
        document = document.lower()
    
        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
    
        documentss.append(document)
    return documentss
    
documents=processed_documents(X)

In [6]:
def vectorization(data):
    vectorizer = CountVectorizer(max_features=500)
    X = vectorizer.fit_transform(data).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(X).toarray()
    tfidfconverter = TfidfVectorizer(max_features=500)
    X = tfidfconverter.fit_transform(data).toarray()
    return X
X=vectorization(documents)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [8]:
def RFclassifier(X_train, X_test, y_train, y_test):
    classifier = RandomForestClassifier(n_estimators=100, random_state=0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    classifier.score(X_test,y_test)
    return classifier

In [9]:
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
print(classifier.fit(X_train, y_train))
y_pred = classifier.predict(X_test)
print(y_pred)
print(classifier.score(X_test,y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("Accuracy : {}".format(accuracy_score(y_test, y_pred)*100))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
['Teaching Skills' 'General' 'Teaching Skills' 'Teaching Skills' 'General'
 'Behaviour towards students' 'Teaching Skills' 'Teaching Skills'
 'Behaviour towards students' 'Teaching Skills' 'Teaching Skills'
 'Behaviour towards students' 'Teaching Skills' 'Teaching Skills'
 'General' 'Teaching Skills' 'General' 'General' 'General'
 'Behaviour towards students' 'General' 'Teaching Skills' 'General'
 'Punctuality' 'General' 'General' 'General' 'Behaviour towards students'
 'Behaviour towards students' 'Teachin

In [10]:
Dclassifier=DecisionTreeClassifier(random_state = 0,max_depth=3, min_samples_leaf=5) 
Dclassifier.fit(X_train, y_train) 
y_pred = Dclassifier.predict(X_test) 
print(y_pred) 
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred)) 
print ("Accuracy : ", accuracy_score(y_test,y_pred)*100)       
print("Report : ", classification_report(y_test, y_pred)) 

['Teaching Skills' 'General' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Punctuality'
 'Teaching Skills' 'Teaching Skills' 'General' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Punctuality' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 '

/Users/simrankumarisah/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
NVclassifier=GaussianNB() 
NVclassifier.fit(X_train, y_train) 
y_pred = NVclassifier.predict(X_test) 
print(y_pred)
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred)) 
print("Report : ", classification_report(y_test, y_pred)) 
print(metrics.accuracy_score(y_test, y_pred)*100)

['Teaching Skills' 'General' 'Teaching Skills' 'Teaching Skills'
 'Behaviour towards students' 'Teaching Skills' 'Teaching Skills'
 'Teaching Skills' 'Behaviour towards students' 'Teaching Skills'
 'Teaching Skills' 'Behaviour towards students' 'Teaching Skills'
 'Punctuality' 'Behaviour towards students' 'Teaching Skills' 'General'
 'Teaching Skills' 'General' 'Behaviour towards students' 'Punctuality'
 'Behaviour towards students' 'General' 'Teaching Skills'
 'Teaching Skills' 'Teaching Skills' 'General'
 'Behaviour towards students' 'Behaviour towards students'
 'Teaching Skills' 'Teaching Skills' 'Teaching Skills' 'Teaching Skills'
 'Behaviour towards students' 'Punctuality' 'Teaching Skills'
 'Teaching Skills' 'General' 'Teaching Skills' 'Teaching Skills'
 'Behaviour towards students' 'Teaching Skills' 'Teaching Skills'
 'General' 'Teaching Skills' 'Punctuality' 'Behaviour towards students'
 'Teaching Skills' 'Behaviour towards students' 'Teaching Skills'
 'Behaviour towards stude

# New Feedback


In [ ]:
feedback=input("Enter new feedback :\t")
e,y=sentimentClassification(feedback)
feedback=pd.DataFrame([{"Data":feedback,"Target":"dummy"}])
feedback = pd.concat([recommendation , feedback]).reset_index(drop=True)
feedback_X, feedback_y = feedback.Data, feedback.Target
feedback_documents=processed_documents(feedback_X)
feedback_X=vectorization(feedback_documents)
feedbackX_train, feedbackX_test, feedbacky_train= feedback_X[:feedback_X.shape[0]-1,:],feedback_X[0,:],feedback_y.iloc[:feedback_y.size-1]
feedback_model = RandomForestClassifier(n_estimators=1000, random_state=0)
feedback_model.fit(feedbackX_train, feedbacky_train)
feedback_prediction = feedback_model.predict(feedbackX_test.reshape(1,-1))
print("..................................................................")
print("Polarity Score : {}".format(y))
print("Comment  : {}".format(e))
print(feedback_prediction)
print("Feedback Classification :{}".format(feedback_classification(feedback_prediction)))

In [ ]:
plot_size = plt.rcParams["figure.figsize"] 
print(plot_size[0]) 
print(plot_size[1])
plot_size[0] = 8
plot_size[1] = 6
plt.rcParams["figure.figsize"] = plot_size
recommendation.Target.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=["red", "yellow", "green","blue"])

In [ ]:
frequency = np.unique(recommendation['Target'] , return_counts=True)

In [ ]:
plt.hist(recommendation['Target'])